<a href="https://colab.research.google.com/github/Trevor16gordon/random-forests-cuda/blob/rakshith_develop2/PyCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
hello


NameError: ignored